In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
df=pd.read_csv('train_dataset.csv')
df.head(10)

,length,diameter,height,whole-weight,shucked-weight,viscera-weight,shell-weight,sex_F,sex_I,sex_M,rings
0,124,97,41,243.8,77.5,50.1,77.0,0,0,1,14
1,109,88,27,183.7,85.8,40.3,47.5,1,0,0,10
2,136,108,38,324.6,143.3,70.8,94.3,0,0,1,12
3,110,88,33,172.1,62.4,33.8,60.0,0,1,0,17
4,78,60,20,53.3,22.1,11.8,16.8,0,1,0,7
5,123,99,31,257.3,87.0,58.6,64.9,0,0,1,11
6,106,82,28,136.2,61.9,28.3,36.7,0,0,1,6
7,105,83,34,166.5,55.1,33.7,62.0,1,0,0,13
8,113,88,25,160.4,71.9,36.5,43.0,0,0,1,9
9,76,55,19,48.5,21.2,9.7,42.0,0,1,0,6


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3550 entries, 0 to 3549
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   length          3550 non-null   int64  
 1   diameter        3550 non-null   int64  
 2   height          3550 non-null   int64  
 3   whole-weight    3550 non-null   float64
 4   shucked-weight  3550 non-null   float64
 5   viscera-weight  3550 non-null   float64
 6   shell-weight    3550 non-null   float64
 7   sex_F           3550 non-null   int64  
 8   sex_I           3550 non-null   int64  
 9   sex_M           3550 non-null   int64  
 10  rings           3550 non-null   int64  
dtypes: float64(4), int64(7)
memory usage: 305.2 KB


In [5]:
df.describe()

,length,diameter,height,whole-weight,shucked-weight,viscera-weight,shell-weight,sex_F,sex_I,sex_M,rings
count,3550.000000,3550.000000,3550.000000,3550.000000,3550.00000,3550.000000,3550.000000,3550.000000,3550.000000,3550.000000,3550.000000
mean,104.649014,81.449296,27.875493,165.574085,71.86462,36.039690,47.641493,0.308451,0.322817,0.368732,9.925915
std,24.143150,19.944770,8.506358,98.768181,44.78615,22.012372,27.932808,0.461919,0.467619,0.482529,3.230225
min,15.000000,11.000000,0.000000,0.400000,0.20000,0.100000,0.300000,0.000000,0.000000,0.000000,1.000000
25%,90.000000,69.000000,23.000000,88.025000,36.80000,18.400000,26.000000,0.000000,0.000000,0.000000,8.000000
50%,109.000000,85.000000,28.000000,159.400000,67.10000,34.000000,46.000000,0.000000,0.000000,0.000000,9.000000
75%,123.000000,96.000000,33.000000,230.875000,100.60000,50.400000,66.000000,1.000000,1.000000,1.000000,11.000000
max,163.000000,130.000000,226.000000,565.100000,297.60000,152.000000,201.000000,1.000000,1.000000,1.000000,29.000000


In [6]:
df.corr().rings.sort_values(ascending=False)

rings             1.000000
shell-weight      0.627542
diameter          0.577902
length            0.560270
height            0.552704
whole-weight      0.544464
viscera-weight    0.508817
shucked-weight    0.426539
sex_F             0.253636
sex_M             0.185109
sex_I            -0.441556
Name: rings, dtype: float64

In [7]:
X=df.drop(columns=['rings'],axis=1)
y=df['rings']

In [8]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import LinearSVR
from sklearn.metrics import mean_squared_error

classifiers=[
    RandomForestRegressor(n_estimators=1000,oob_score=True),
    LinearRegression(normalize=True),
]
for classifier in classifiers:
    model=classifier.fit(X_train,y_train)
    y_predicted=model.predict(X_test)
    print(f'{model.__class__.__name__} got error {mean_squared_error(y_test,y_predicted)}')
    print(classifier.get_params())

RandomForestRegressor got error 4.206352774647887
{'bootstrap': True, 'ccp_alpha': 0.0, 'criterion': 'mse', 'max_depth': None, 'max_features': 'auto', 'max_leaf_nodes': None, 'max_samples': None, 'min_impurity_decrease': 0.0, 'min_impurity_split': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'min_weight_fraction_leaf': 0.0, 'n_estimators': 1000, 'n_jobs': None, 'oob_score': True, 'random_state': None, 'verbose': 0, 'warm_start': False}
LinearRegression got error 4.107532404510068
{'copy_X': True, 'fit_intercept': True, 'n_jobs': None, 'normalize': True, 'positive': False}


In [26]:
from sklearn.ensemble import BaggingRegressor
bag_reg=BaggingRegressor(base_estimator=LinearRegression(normalize=True),n_estimators=2000,max_samples=100,bootstrap=False,n_jobs=-1)
bag_reg.fit(X_train,y_train)
y_predicted=bag_reg.predict(X_test)
print(f'{bag_reg.__class__.__name__} got error {mean_squared_error(y_test,y_predicted)}')

BaggingRegressor got error 4.132841132798063


In [27]:
test=pd.read_csv('test_dataset.csv')
test.head(10)

,length,diameter,height,whole-weight,shucked-weight,viscera-weight,shell-weight,sex_F,sex_I,sex_M
0,111,87,29,139.5,52.4,31.5,48.0,0,1,0
1,70,53,19,39.8,14.6,9.8,12.0,0,1,0
2,77,57,17,48.8,24.3,8.9,13.6,0,1,0
3,122,99,37,230.6,107.2,58.1,49.0,1,0,0
4,115,95,32,222.8,99.1,54.9,58.0,0,0,1
5,123,103,34,228.0,86.1,44.9,84.0,1,0,0
6,121,97,32,240.2,83.4,57.5,76.0,1,0,0
7,122,98,34,269.5,140.9,50.0,60.9,1,0,0
8,93,72,25,87.3,33.8,21.5,29.0,0,0,1
9,111,89,35,229.3,110.2,48.8,55.7,1,0,0


In [29]:
results=bag_reg.predict(test)
submission=pd.DataFrame(results,columns=['rings'])
submission.to_csv('submission.csv',index=False)